# **Connect to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/My\ Drive/Colab\ Notebooks/

/content/gdrive/My Drive/Colab Notebooks


# **Load dataset**

In [ ]:
import numpy as np
from sklearn.utils import shuffle
import h5py

### **Load training set**

In [ ]:
def load_train_dataset():
    X_train = []
    y_train = []

    filename_prefix = 'train_data/X_train_'

    for i in range(1, 13):
        
        filename = filename_prefix + str(i) + '.hdf5'
        print('Loading batch ' + str(i) + ' ...')

        with h5py.File(filename, 'r') as f:
            for j in f['X'][:]:
                X_train.append(j)

            for j in f['y'][:]:
                y_train.append(j)

    return X_train, y_train

In [ ]:
X_train, y_train = load_train_dataset()

Loading batch 1 ...
Loading batch 2 ...
Loading batch 3 ...
Loading batch 4 ...
Loading batch 5 ...
Loading batch 6 ...
Loading batch 7 ...
Loading batch 8 ...
Loading batch 9 ...
Loading batch 10 ...
Loading batch 11 ...
Loading batch 12 ...


In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(227755, 7500)
(227755, 1)


In [ ]:
X_train, y_train = shuffle(X_train, y_train)

### **Load test set**
Note: Load after training process completed to save memory

In [ ]:
def load_test_dataset():
    X_test = []
    y_test = []

    filename_prefix = 'test_data/X_test_'

    for i in range(1, 6):
        
        filename = filename_prefix + str(i) + '.hdf5'
        print('Loading batch ' + str(i) + ' ...')

        with h5py.File(filename, 'r') as f:
            for j in f['X'][:]:
                X_test.append(j)

            for j in f['y'][:]:
                y_test.append(j)

    return X_test, y_test

In [ ]:
X_test, y_test = load_test_dataset()

Loading batch 1 ...
Loading batch 2 ...
Loading batch 3 ...
Loading batch 4 ...
Loading batch 5 ...


In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
print(X_test.shape)
print(y_test.shape)

(89754, 7500)
(89754, 1)


In [ ]:
X_test, y_test = shuffle(X_test, y_test)

# **Building Model ANN**

### **Setting Model**

**The sigmoid function**

In [ ]:
def sigmoid(x):
    # Activation function: sigmoid function
    return 1.0 / (1 + np.exp(-x))

**The derivative of the sigmoid function**

In [ ]:
def derivative_sigmoid(x):
    # Derivative of the sigmoid function
    return x * (1 - x)

**Creates minibatch data provided to the model**

In [ ]:
def next_batch(X, y, batch_size):
    # Mini-batch
    for i in np.arange(0, X.shape[0], batch_size):
        yield X[i:i + batch_size], y[i:i + batch_size]

**Build model Multilayer Perceptron**

In [ ]:
class NeuralNetwork:
    """ Neural network with backpropagation """

    def __init__(self, layers, alpha=0.05):
        # A list which represents the architecture of the model
        # Ex: [2, 2, 1] - first layer has 2 nodes, hidden layer has 2 nodes, output layer has 1 node
        self.W = []
        self.layers = layers
        self.alpha = alpha
        self.loss_list = []

        # Initialize weight (from the first layer, stop before we reach the last 2 layers)
        for i in np.arange(0, len(layers) - 2):
            # Randomly initialize a weight matrix, in each layer, add an extra node for the bias
            w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
            self.W.append(w / np.sqrt(layers[i]))

        # Initialize weight for the last 2 layers
        w = np.random.randn(layers[-2] + 1, layers[-1])
        self.W.append(w / np.sqrt(layers[-2]))

    def __repr__(self):
        return "Neural Network: {}".format("-".join(str(l) for l in self.layers))

    def fit(self, X, y, epochs=20, display_update=1, batch_size=8):
        # Bias trick
        X = np.c_[X, np.ones((X.shape[0]))]

        for epoch in np.arange(0, epochs):
            epoch_loss = []

            for (batchX, batchY) in next_batch(X, y, batch_size):
                self.fit_partial(batchX, batchY)

                epoch_loss.append(self.calculate_loss(batchX, batchY))

            loss = np.average(epoch_loss)
            self.loss_list.append(loss)

            # Display the training update
            if epoch == 0 or (epoch + 1) % display_update == 0:
                print("[INFO] epoch={}, loss={:.7f}".format(epoch + 1, loss))

    def fit_partial(self, x, y):
        # Construct a list of output activations for each layer
        A = [np.atleast_2d(x)]

        # ------------
        # FEEDFORWARD
        # ------------
        for layer in np.arange(0, len(self.W)):
            # Net input
            net = A[layer].dot(self.W[layer])

            # Activation function for net input
            out = sigmoid(net)

            A.append(out)

        # ----------------
        # BACKPROPAGATION
        # ----------------
        error = A[-1] - y

        # Construct a list of layer times the derivative of our activation function
        D = [error * derivative_sigmoid(A[-1])]

        for layer in np.arange(len(A) - 2, 0, -1):
            delta = D[-1].dot(self.W[layer].T)
            delta = delta * derivative_sigmoid(A[layer])
            D.append(delta)

        # Reverse order of the D
        D = D[::-1]

        # --------------------
        # WEIGHT UPDATE PHASE
        # --------------------
        for layer in np.arange(0, len(self.W)):
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

    def calculate_loss(self, X, targets):
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, add_bias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)

        return loss

    def predict(self, X, add_bias=True):
        p = np.atleast_2d(X)

        if add_bias:
            p = np.c_[p, np.ones((p.shape[0]))]

        for layer in np.arange(0, len(self.W)):
            p = sigmoid(np.dot(p, self.W[layer]))

        return p

### **Impliment Model**

**Model with parameters:**
- 2 hidden layer: 
  + 1st: 512 nodes 
  + 2nd: 512 nodes
- batch_size: 8
- learning rate: 0.05
- epoch: 20

In [ ]:
# Train the network
print("[INFO] training network...")
nn = NeuralNetwork([X_train.shape[1], 512, 512, 1])
print("[INFO] {}".format(nn))
nn.fit(X_train, y_train)

[INFO] training network...
[INFO] Neural Network: 7500-512-512-1
[INFO] epoch=1, loss=0.4940480
[INFO] epoch=2, loss=0.4738015
[INFO] epoch=3, loss=0.4646414
[INFO] epoch=4, loss=0.4552187
[INFO] epoch=5, loss=0.4382529
[INFO] epoch=6, loss=0.4323542
[INFO] epoch=7, loss=0.4307431
[INFO] epoch=8, loss=0.4280023
[INFO] epoch=9, loss=0.4253564
[INFO] epoch=10, loss=0.4213176
[INFO] epoch=11, loss=0.4205356
[INFO] epoch=12, loss=0.4175257
[INFO] epoch=13, loss=0.4172338
[INFO] epoch=14, loss=0.4149153
[INFO] epoch=15, loss=0.4128013
[INFO] epoch=16, loss=0.4122982
[INFO] epoch=17, loss=0.4118650
[INFO] epoch=18, loss=0.4111830
[INFO] epoch=19, loss=0.4094522
[INFO] epoch=20, loss=0.4072834


### **Evaluate model**

In [ ]:
predictions = nn.predict(X_test)

**Confusion matrix**

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, np.round(predictions))
print(cm)

[[54541  7812]
 [ 9957 17444]]


**Accuracy**

In [ ]:
TP, FP, FN, TN = cm[0][0], cm[0][1], cm[1][0], cm[1][1]

In [ ]:
accuracy = (TP + TN) / (TP + TN + FP + FN)
print(accuracy)

0.8020255364663413


### **Output parameters of weght and loss**
Predict the new data without traning again 

In [ ]:
def output_h5py(file_name, w):
    file = h5py.File(file_name, 'w')
    file.create_dataset('data', data=w)
    file.close()

In [ ]:
output_h5py('mlp_v2/weight_0.hdf5', nn.W[0])
output_h5py('mlp_v2/weight_1.hdf5', nn.W[1])
output_h5py('mlp_v2/weight_2.hdf5', nn.W[2])

In [ ]:
output_h5py('mlp_v2/loss.hdf5', np.array(nn.loss_list))